In [1]:
import pandas as pd

from pathlib import Path

/Users/jonathanlim/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [3]:
PROJ_DIR = Path.cwd().parent
DATA_DIR = PROJ_DIR / 'data'
LOG_DIR = PROJ_DIR / 'logs'

LOG_DIR.mkdir(parents=True, exist_ok=True) 
LOG_SAVE_DIR = LOG_DIR / 'attn_lstm'

In [4]:
master_df = pd.read_pickle(DATA_DIR / 'full_master_updated.pkl')
val_df = pd.read_csv(str(LOG_SAVE_DIR / 'val_performance.csv'))

## Generate Knowledge Extraction metrics based on best Key-clip Prediction model's predictions

In [5]:
def best_keyclip_prediction_true_positives(row):
    return row['true_positives'] if row['IsPredUseful'] == 1 else 0

def generate_metrics_from_master_and_val_df(master_df, val_df, master_key='Sentence', val_key='text'):
  combined_df = pd.merge(master_df, val_df, left_on=master_key, right_on=val_key, how='inner')
  combined_df['best_kc_pred_true_positives'] = combined_df.apply(lambda row: best_keyclip_prediction_true_positives(row), axis=1)
  total_num_gold = combined_df['num_gold'].sum()
  total_num_predicted = combined_df['num_predicted'].sum()
  total_true_pos = combined_df['best_kc_pred_true_positives'].sum()
  precision = round(total_true_pos / total_num_predicted, 2)
  recall = round(total_true_pos / total_num_gold, 2)
  f1 = round(2*(precision*recall) / (precision+recall), 2)

  print(f"Total true positives (predicted words overlap with gold words): {total_true_pos}")
  print(f"Total predicted words: {total_num_predicted}")
  print(f"Total gold words: {total_num_gold}")
  print(f"Precision: {precision}, Recall: {recall}, F1: {f1}")
  
  return precision, recall, f1

In [7]:
def get_words(sentence, num=5):
    return " ".join(sentence.split(" ")[:num])

In [8]:
master_df['five_words'] = master_df['Sentence'].apply(lambda x: get_words(x, num=5))

In [9]:
val_df['five_words'] = val_df['summary'].apply(lambda x: get_words(x, num=5))

In [11]:
generate_metrics_from_master_and_val_df(master_df, val_df, master_key='five_words', val_key='five_words')

Total true positives (predicted words overlap with gold words): 2057
Total predicted words: 7434
Total gold words: 10439
Precision: 0.28, Recall: 0.2, F1: 0.23


(0.28, 0.2, 0.23)